In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'NanumGothicCoding'
plt.rcParams['font.size'] = 10
import seaborn as sns

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error  
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import precision_score 
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

***
스태킹(Stacking)은 베이스 학습기와 메타 학습기로 구성되어 있고, 베이스 학습기와 메타 학습기는 서포트 벡터 머신, 랜덤 포레스트와 같은 학습 모델이다.  
베이스 학습기가 먼저 학습한 후, 메타 학습기는 베이스 학습기의 예측 피쳐 데이터로 활용해 최종 예측을 한다.  
***

***
암 데이터 사용해 암 여부를 예측하는 모델을 생성, 학습한다.
***

In [2]:
# 데이터 불러오기
raw_data = datasets.load_breast_cancer() # 사이킷런이 제공하는 암 데이터를 불러온다.

# 피쳐, 레이블 데이터 저장
xData = raw_data.data # 피쳐 데이터를 저장한다.
yData = raw_data.target # 피쳐 데이터에 따른 레이블을 저장한다.
print(xData.shape, yData.shape)


# 학습 데이터와 테스트 데이터로 분할
x_train, x_test, y_train, y_test = train_test_split(xData, yData, random_state=0)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

# 데이터 표준화
std_scale = StandardScaler() # 표준화 스케일러 객체를 만든다.
x_train = std_scale.fit_transform(x_train) # 학습 데이터를 스케일러로 표준화하고 적용한다.
x_test = std_scale.transform(x_test) # 테스트 데이터를 학습 데이터로 표준화된 스케일러에 적용한다.

# 모델 생성 후 데이터 학습
from sklearn.ensemble import AdaBoostClassifier # 에이다 부스트 알고리즘 사용

# 모델을 만든다.
clf = AdaBoostClassifier()

# 표준화된 학습 데이터 x_train와 학습 데이터에 따른 레이블 데이터 y_train으로 앙상블 에이다 부스트 모델을 학습시킨다.
clf.fit(x_train, y_train)


(569, 30) (569,)
(426, 30) (143, 30) (426,) (143,)


AdaBoostClassifier()

***
모델 생성 후 학습
***

In [3]:
# 베이스 학습기
from sklearn.svm import SVC # 서포트 벡터 머신 알고리즘 사용
from sklearn.naive_bayes import GaussianNB # 가우시안 나이브베이즈 알고리즘 사용

clf1 = SVC() # 서포트 벡터 머신 알고리즘 학습기 모델 객체
clf2 = GaussianNB() # 가우시안 나이브베이즈 학습기 모델 객체

# 메타 학습기
from sklearn.linear_model import LogisticRegression # 로지스티 회귀 알고리즘 사용

clf3 = LogisticRegression()

# 앙상블 스테킹
from sklearn.ensemble import StackingClassifier # 스테킹 알고리즘 사용

#estimators 속성에 스테킹 모델에서 사용할 베이스 학습기를 지정하고, final_estimator 속성에 스테킹 모델에서 사용할 메타 학습기를 지정해서 앙상블 스테킹 모델을 만든다.
clf = StackingClassifier(estimators=[('svm', clf1), ('gnb', clf2)], final_estimator=clf3)

# 표준화된 학습 데이터 x_train와 학습 데이터에 따른 레이블 데이터 y_train으로 앙상블 에이다 부스트 모델을 학습시킨다.
clf.fit(x_train, y_train)


StackingClassifier(estimators=[('svm', SVC()), ('gnb', GaussianNB())],
                   final_estimator=LogisticRegression())

***
학습된 모델로 테스트 데이터를 예측한다.
***

In [4]:
#predict() 메소드의 인수로 표준화된 테스트 데이터를 넘겨서 예측한다.
predict = clf.predict(x_test)
print(predict)

[0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 0 1 0 1 1 0 1 1 1 0 0 0 0 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 0 1 1 0 1 0
 0 1 1 1 1 1 0 0 0 1 0 1 1 1 0 0 1 1 0 0 1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 0 1
 0 0 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0]


***
학습된 모델을 평가한다
***

In [5]:
# 혼동 행렬
conf_matrix = confusion_matrix(y_test, predict)
print(conf_matrix)

[[50  3]
 [ 2 88]]


In [6]:
#분류 리포트
class_report = classification_report(y_test, predict, target_names=raw_data.target_names)
print(class_report)

              precision    recall  f1-score   support

   malignant       0.96      0.94      0.95        53
      benign       0.97      0.98      0.97        90

    accuracy                           0.97       143
   macro avg       0.96      0.96      0.96       143
weighted avg       0.96      0.97      0.96       143

